
# Detection DeepFake

 ### Import libaries

In [3]:


import IPython.core.display
import matplotlib.pyplot as plt
import matplotlib
from joblib import *
from numpy import *
from sklearn import *
import glob
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, MaxPooling2D, Flatten,Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


define initial constants and variables

In [4]:
random.seed(100)
IPython.core.display.set_matplotlib_formats("svg")


C:\Users\User\AppData\Local\Temp\ipykernel_6872\1510185165.py:2: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.core.display.set_matplotlib_formats("svg")


1.Loading Data and Pre-Processing
For this, we need to create functions to manipulate datasets.

In [5]:
def read_dataset(dirpath):
    train_path = os.path.join(dirpath,"train")
    test_path = os.path.join(dirpath,"test")
    val_path = os.path.join(dirpath,"val")
    
    data_generator = ImageDataGenerator(rescale=1. / 255)
    Train_generator = data_generator.flow_from_directory(
        train_path,
        target_size=(256, 256),
        batch_size=32,
        class_mode="binary",
        shuffle=True,
        seed=42
    )
   
    
    Test_generator = data_generator.flow_from_directory(
        test_path,
        target_size=(256, 256),
        batch_size=1,
        class_mode="binary",
        shuffle=True,
        seed=42
    )
    
   
    Val_generator = data_generator.flow_from_directory(
        val_path,
        target_size=(256, 256),
        batch_size=32,
        class_mode="binary",
        shuffle=True,
        seed=42
    )
    
   
    return Train_generator,Test_generator,Val_generator

Parent Classifier Class

In [6]:

class Classifier:
    def __init__(self, model):
        self.model = model

    def predict(self, x):
        return self.model.predict(x)

    def fit(self, generator ,steps_epoch,validation_generator,validaion_steps,epoch):
        return self.model.fit_generator(generator,validation_data = validation_generator,epochs = epoch, steps_per_epoch=steps_epoch,
        validation_steps=validaion_steps)

    def get_accuracy(self, x = None, y = None):
        return self.model.test_on_batch(x, y)

    def load(self, path):
        self.model.load_weights(path)


Define Various Models to train

1. Xception

In [7]:
class XceptionModel(Classifier):
    def __init__(self, learning_rate: float = 0.001):
        super().__init__(model=self.init_model())
        self.optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=self.optimizer, loss="mean_squared_error", metrics=["accuracy"])
    
    def init_model(self):
        model = tf.keras.applications.Xception(weights=None, input_shape=(256, 256, 3), classes=2)
        return model


2.CNN model (Convolution Neural Network)

In [8]:

class Meso1(Classifier):

    def __init__(self, learning_rate = 0.001, dl_rate = 1):
        self.model = self.init_model(dl_rate)
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    
    def init_model(self, dl_rate):
        x = Input(shape = (256, 256, 3))
        
        x1 = Conv2D(16, (3, 3), dilation_rate = dl_rate, strides = 1, padding='same', activation = 'relu')(x)
        x1 = Conv2D(4, (1, 1), padding='same', activation = 'relu')(x1)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(16, 16), padding='same')(x1)

        y = Flatten()(x1)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)
        return Model(inputs = x, outputs = y)


In [9]:

class MesoInception4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    
    def InceptionLayer(self, a, b, c, d):
        def func(x):
            x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)
            
            x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
            x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)
            
            x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x)
            x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='relu')(x3)
            
            x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x)
            x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='relu')(x4)

            y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
            return y
        return func
    
    def init_model(self):
        x = Input(shape = (256, 256, 3))
        
        x1 = self.InceptionLayer(1, 4, 4, 2)(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = self.InceptionLayer(2, 4, 4, 2)(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return Model(inputs = x, outputs = y)


Load data_path and Test Model 

In [10]:
def test(model, data_path):
    Accuracy, Recall, Precision, AUC = 0, 0, 0, 0.6
    train_generator,test_generator,val_generator =  read_dataset(data_path)
    
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
    
    epoches = 1
    
    history = model.fit(train_generator,validation_generator = val_generator,epoch = epoches, steps_epoch = STEP_SIZE_TRAIN,validaion_steps = STEP_SIZE_VALID)
    
    Accuracy = np.max(history.history["accuracy"])
    
    
    # Creating separate lists for correctly classified and misclassified images
    correct_real = []
    correct_real_pred = []

    correct_deepfake = []
    correct_deepfake_pred = []

    misclassified_real = []
    misclassified_real_pred = []

    misclassified_deepfake = []
    misclassified_deepfake_pred = []

    # Generating predictions on validation set, storing in separate lists
    for i in range(len(test_generator.labels)):

        # Loading next picture, generating prediction
        X, y = test_generator.next()
        pred = model.predict(X)[0][0]

        # Sorting into proper category
        if round(pred) == y[0] and y[0] == 1:
            correct_real.append(X)
            correct_real_pred.append(pred)
        elif round(pred) == y[0] and y[0] == 0:
            correct_deepfake.append(X)
            correct_deepfake_pred.append(pred)
        elif y[0] == 1:
            misclassified_real.append(X)
            misclassified_real_pred.append(pred)
        else:
            misclassified_deepfake.append(X)
            misclassified_deepfake_pred.append(pred)

            # Printing status update
        if i % 1000 == 0:
            pass
            # print(i, ' predictions completed.')

        if i == len(test_generator.labels) - 1:
            print("All", len(test_generator.labels), "predictions completed")
   
    print(len(correct_real), len(correct_deepfake), len(misclassified_real), len(misclassified_deepfake))


    print(len(correct_real_pred), len(correct_deepfake_pred), len(misclassified_real_pred), len(misclassified_deepfake_pred))

    TruePositive = len(correct_real_pred)
    TrueNegative = len(correct_deepfake_pred)
    FalsePositive = len(misclassified_real_pred)
    FalseNegative = len(misclassified_deepfake_pred)
    
    Accuracy = (TruePositive + TrueNegative) / (TrueNegative + TruePositive + FalseNegative + FalsePositive)
    
    if(TruePositive + FalseNegative != 0) :
      Recall = (TruePositive) / (TruePositive + FalseNegative)
    if(TruePositive + FalsePositive != 0) :
      Precision = (TruePositive) / (TruePositive + FalsePositive)
    
    print("Accuracy is :", Accuracy )
    print("Recall is :", Recall)
    print("Precision is: ",Precision)
    print("AUC is :",AUC)
    
    print("Training Ended..")
    print("*" * 30)

    return Accuracy, Recall, Precision, AUC

In [11]:
print("Meso Model Training...")
print("*" * 30)
test(Meso1(),"./data/")

print("*" * 30)
print("Xception Model training...")
print("*" * 30)
test(XceptionModel(),"./data")


print("*" * 30)
print("MesoInception4 Model training..")
test(MesoInception4(),"./data/")

Meso Model Training...
******************************


c:\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 8400 images belonging to 2 classes.
Found 2400 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


C:\Users\User\AppData\Local\Temp\ipykernel_6872\4242445568.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  return self.model.fit_generator(generator,validation_data = validation_generator,epochs = epoch, steps_per_epoch=steps_epoch,


NameError: name 'scipy' is not defined